In [ ]:
import tensorflow as tf
print(tf.__version__)

[slides](https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/blob/master/Lectures/xNNs_070_Implementation.pdf)

[disk](file:///F:/Data/xNNs_070_Implementation.pdf)

# Implementation Notes

## Hardware and Software

Software can be more general (hardware agnostic) that leans on runtime intelligent hardware to optimize the perforance of the program duting execution. 

Software can rely on compile time intelligent hardware. 

### __Graph Specification__ 
Is a representation of a high-level hardware agnostic computer algorithm. nodes are operators. Implicit data and instruction movement. Edges prepresent dependenies (memory)

### __Graph Lowering__ 
Map a high level program to a low-level, hardware specific graph. Includes compute and communication nodes, as well as data and instruction edges. Nodes map 1 to 1 to hardware.

This is an iterative optimization process that can be improved with knowledge of domain and hardware.

### __Graph Execution__ 
Software runtime runs on a control processor and cycles through nodes on the low level graph. Hardware execures the nodes with computation and communication primitives as well as general compute for everything else.

## Optimized XNN Design

Some hardware aware design considerations:
- Operator selection: quantization, sparisification, and compression
- Network sizing: depth, width, input size

### __Quantization__ 
__Quantization__ reduces the number of bits required to represent feature maps and parameters. 

Quantizing deep convolutional networks for efficient inference: A whitepaper
https://arxiv.org/abs/1806.08342

*__Per-channel quantization of weights and per-layer quantization of activations to 8-bits of precision__ post-training produces classification accuracies within 2% of floating point networks for a wide variety of CNN architectures. __Model sizes can be reduced by a factor of 4 by quantizing weights to 8-bits__, even when 8-bit arithmetic is not supported...Observe a speedup of 2x-3x for quantized implementations compared to floating point on CPUs*


Reduces memory and communication requirements of the network. Multiplication complexity scales to the square of the number of bits, while addition scales linearly with the number of bits.

32 bit float, 16 bit float, 8 bit fixed (common for CNN inference), and 4 bit fixed

### __Sparsification__

__Sparsifiaton__ Reduces memory and the required number of computations. Random sparsity can be encouraged with L1 regularization as well as thresholding dense coefficient values.

Network pruning methods exist. Using second oreder derivatives of parameters wrt error function

https://openreview.net/pdf?id=Sy1iIDkPM

### Appropriate Sizing
Match the network size to the available memory on device.

### Train Test Constraints

__Train__
- Can batch inputs
- Need extra batch norm computations (moving average compute)
- Need error calculation and extra memory for backprop
- Need higher precision floating point

__Test__
- Absorb batch norm op into convolution
- Need network output
- Less memory space required since no backprop
- OK to use lower precision in fixed points

## XNN Software

Software maps the XNN design to hardware. Does graph specification, lowering and execution. User specifies the network graph design and software automatically creates a reverse graph to propagate errors to weights.

### __CNN Graph Lowering__

• Domain agnostic hardware agnostic optimization
    - Remove unneeded edges and nodes required for specified input output
    - Constant folding and constant propagation
    
• Domain specific hardware agnostic optimization
    - xNNs: remove dropout and scale associated weight layers
    - xNNs: absorb batch norm into convolution and create a bias term

• Domain specific hardware aware optimization
    - xNNs: transform data layouts (tensor ordering)
    - xNNs: node fusion, tiling and grouping
    - xNNs: post training quantization

• Domain agnostic hardware specific code generation
    - Memory planning for all tensors
    - Data movement and compute strategy selection for each node
    - Code generation for selected strategy

### __Graph Execution__
role of the software runtime. Ties addresses for dynamic tensors into the graph.

During execution cycle through the nodes, sing the appropriate communication and computation primitives.

## XNN Hardware

__Dennard Scaling__ Transistor power is no longer proportional to area. More transistors increases power consumption increases heat...Now we need good architecture designs to advance performance,

__Dark Silicon__ only a fraction of the transistors on a chip can be activated at one time (or they will melt) -> design accelerators to be really efficient.

__Dark Memory__ only a fraction of DRAM can be active at one time -> maximize data locality to minimize data movement and boost performance



What consumes power (most to least)
- Physical movement (motors)
- Long distance communication
- off device comm.
- on device comm.
- computation

__How to design optimal hardware:__ 
- minimize off dev comm. -> inlude a lot of on dev memory
- minimize on dev movememnt -> data locality
- optimize comptute -> match accelerators to algorithm

### Domain Specific Architecture (DSA)
 
DSAs are optimized for a specific application. Includes memory, control, communication and compute. Can vastly improve compute performance.

__DSAs are different than CPUs__ 

They are designed to optimize throughput, Domain optimized communication, comput, and mem.

__Primitive defined domains__ 

Define the domian in terms of fundamental math, not an application

#### __Memory__ 
Need enough on device memory such that either all feature maps or all coefficients fit on device.

 If all feature maps can be loaded to on device memory you can perform the matrix multiply by grabbing the coefficients by rows and performin the op. (likewise for if coefficients can be stored on-device)
 
 If neither fit entirely in memory, than the chip has to load both in groups (row/col) creating a small section of the output at one time. Will also require one of either coeefs or filters to be loaded on to-device more than once. (double loop situation)
 
__Design the memory to have enough to store feature maps fully on device so only weights need to be moved.__ As you progress through the network, grab weights for each layer.

<img src="../img/on-dev-mem.PNG">


#### Control

Control cycles through the nodes on the low-level graph. Gives instructions to the appropriate primitive. Typucally a general compute resource that executes all non primitive supported nodes

#### Communication Strategy

Ping-pong buffers in DRAM and local mem to allow concurrent compute and communication. 

Performs the following in parallel:
- external <-> local
- local <-> computer
- Computation

__External bandwidth is much less than internal bandwidth__ -> need high data reuse or keep some data on device at all times.

<img src="../img/communication.PNG" width="300">


__Communication Primitive__

A transform primitive with a communication framework. 
- Gather: vector read on-dev/DRAM
- Transform: compress/decompress
- Transform: encrypt/decrypt
- Scatter: vector write on-mem/DRAM

#### Computation Strategy

Create reaaly efficient primitives for operations that are done alot (max, min, sort, rank, pool, find, median) Everything else performed on a general compute. This allows new  operations.

Use a matrix comp primitice for (conv, fft)

Reads and writes are adress aligned to maximize bandwidth efficiency. Pre/post-processing formats data to to transform a comatibale problem to matrix matrix multiplication. Ping-pong registers based on matrix mult method


__Multiply Matrices w < $N^3$ MACs__

Multiply costs more than addition.-> create intermediary addition terms that substtute multiplication

__Strassen's Algorithm__


In [ ]:
matr_A = [[1,2],[3,4]]
matr_B = [[5,6],[7,8]]

def matmul(matrix_A, matrix_B):
    assert(len(matrix_A[0])==len(matrix_B))
    out_height = len(matrix_A)
    interm_dim = len(matrix_A[0])
    out_width = len(matrix_B[0])
    
    matrix_C = [[0 for j in range(out_width)] for i in range(out_height)]
    for h in range(out_height):
        for w in range(out_width):
            c = 0
            for k from range(interm_dim):
                c += matrix_A[h][k]*matrix_B[k][w]
            C[h][w] = c
    return C
            

def strassens_matmul(matrix_A, matrix_B):
    assert(len(matrix_A[0])==len(matrix_B))
    out_height = len(matrix_A)
    interm_dim = len(matrix_A[0])
    out_width = len(matrix_B[0])
    
    matrix_C = [[0 for j in range(out_width)] for i in range(out_height)]
    
    S_1 = 

## XNN Performance